# Bollinger Bands Strategy with Backtrader Library

This notebook implements the same Bollinger Bands trading strategy using the `backtrader` library for comparison and validation.

## Strategy Overview
- **Entry Signal**: Long when close < lower band, Short when close > upper band
- **Exit Signal**: Close position when price reaches middle band
- **Spread Cost**: Fixed 5 pips per trade
- **Position Limit**: Maximum 1 position at a time

## Configuration and Imports

In [19]:
# Install required packages
import subprocess
import sys

packages = ['backtrader', 'matplotlib']
for package in packages:
    try:
        __import__(package)
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed successfully")

✅ backtrader already installed
✅ matplotlib already installed


In [20]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime, timedelta

# Backtrader imports
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeeds

# Suppress warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


## Configuration Parameters

In [21]:
# ========================================
# 📋 CONFIGURATION PARAMETERS
# ========================================

# 📁 Data Configuration
TICK_DATA_PATH = '/home/edocame/Desktop/data_python/EURCHF-2025-02.csv'

# 📊 Bollinger Bands Strategy Parameters
BB_WINDOW = 1000       # Period for Bollinger Bands calculation (minutes)
BB_STD = 2             # Number of standard deviations for bands

# 🎯 Trading Strategy Parameters
INITIAL_CAPITAL = 10000    # Starting capital
COMMISSION_PIPS = 5        # Fixed spread cost per trade in pips
PIP_VALUE = 10000          # Pip multiplier for EURCHF (1 pip = 0.0001)

# 📈 Performance Analysis Parameters
PLOT_FIGURES_SIZE = (15, 10)

print("✅ Configuration parameters loaded:")
print(f"📁 Tick Data Path: {TICK_DATA_PATH}")
print(f"📊 BB Window: {BB_WINDOW} periods")
print(f"📊 BB Standard Deviation: {BB_STD}")
print(f"💰 Initial Capital: ${INITIAL_CAPITAL:,}")
print(f"📊 Commission: {COMMISSION_PIPS} pips per trade")
print("=" * 50)

✅ Configuration parameters loaded:
📁 Tick Data Path: /home/edocame/Desktop/data_python/EURCHF-2025-02.csv
📊 BB Window: 1000 periods
📊 BB Standard Deviation: 2
💰 Initial Capital: $10,000
📊 Commission: 5 pips per trade


## Data Loading and Preprocessing

In [22]:
def load_and_prepare_data(file_path):
    """
    Load tick data and convert to backtrader format
    """
    print(f"📁 Loading data from: {file_path}")
    
    # Load tick data
    df_ticks = pd.read_csv(file_path, names=['symbol', 'timestamp', 'bid', 'ask'])
    df_ticks['timestamp'] = pd.to_datetime(df_ticks['timestamp'], format='%Y%m%d %H:%M:%S.%f')
    
    # Calculate mid price
    df_ticks['mid_price'] = (df_ticks['bid'] + df_ticks['ask']) / 2
    
    print(f"📊 Loaded {len(df_ticks):,} tick records")
    print(f"📅 Data range: {df_ticks['timestamp'].min()} to {df_ticks['timestamp'].max()}")
    
    # Resample to 1-minute OHLC for backtrader
    df_ticks.set_index('timestamp', inplace=True)
    
    # Create OHLC from mid price
    ohlc_data = df_ticks['mid_price'].resample('1min').ohlc()
    ohlc_data.dropna(inplace=True)
    
    # Rename columns to match backtrader format
    ohlc_data.columns = ['open', 'high', 'low', 'close']
    
    # Add volume (dummy data)
    ohlc_data['volume'] = 1000
    
    print(f"📊 Resampled to {len(ohlc_data):,} 1-minute bars")
    print(f"📅 OHLC range: {ohlc_data.index.min()} to {ohlc_data.index.max()}")
    
    return ohlc_data

# Load data
data = load_and_prepare_data(TICK_DATA_PATH)
print(f"\n📊 Data preview:")
print(data.head())
print(f"\n📊 Data info:")
print(data.info())

📁 Loading data from: /home/edocame/Desktop/data_python/EURCHF-2025-02.csv
📊 Loaded 9,646,673 tick records
📅 Data range: 2025-02-02 22:00:00.138000 to 2025-02-28 21:59:56.709000
📊 Resampled to 28,739 1-minute bars
📅 OHLC range: 2025-02-02 22:00:00 to 2025-02-28 21:59:00

📊 Data preview:
                         open      high       low     close  volume
timestamp                                                          
2025-02-02 22:00:00  0.936815  0.942985  0.936795  0.939735    1000
2025-02-02 22:01:00  0.939700  0.939700  0.937825  0.938385    1000
2025-02-02 22:02:00  0.938380  0.938380  0.937560  0.937560    1000
2025-02-02 22:03:00  0.937555  0.938320  0.937555  0.938060    1000
2025-02-02 22:04:00  0.938065  0.938130  0.936970  0.936970    1000

📊 Data info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28739 entries, 2025-02-02 22:00:00 to 2025-02-28 21:59:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   o

## Bollinger Bands Strategy Implementation

In [23]:
class BollingerBandsStrategy(bt.Strategy):
    """
    Bollinger Bands Mean Reversion Strategy for Backtrader
    
    Entry:
    - Long when price closes below lower band
    - Short when price closes above upper band
    
    Exit:
    - Close long when price reaches middle band
    - Close short when price reaches middle band
    """
    
    # Strategy parameters
    params = (
        ('bb_period', BB_WINDOW),
        ('bb_devfactor', BB_STD),
    )
    
    def __init__(self):
        # Calculate Bollinger Bands using backtrader indicators
        self.bband = btind.BollingerBands(
            self.data.close, 
            period=self.params.bb_period,
            devfactor=self.params.bb_devfactor
        )
        
        # Aliases for readability
        self.upper_band = self.bband.lines.top
        self.middle_band = self.bband.lines.mid  
        self.lower_band = self.bband.lines.bot
        
        # Track trade results
        self.trade_count = 0
        self.profit_pips = 0
        
        print(f"✅ Bollinger Bands initialized: Period={self.params.bb_period}, DevFactor={self.params.bb_devfactor}")
    
    def next(self):
        # Get current price and band values
        price = self.data.close[0]
        upper = self.upper_band[0]
        lower = self.lower_band[0]
        middle = self.middle_band[0]
        
        # Skip if not enough data for bands calculation
        if len(self.data) < self.params.bb_period:
            return
        
        # Exit conditions (close at middle band)
        if self.position:
            if (self.position.size > 0 and price >= middle) or \
               (self.position.size < 0 and price <= middle):
                self.close()
        
        # Entry conditions (no position and price outside bands)
        elif not self.position:
            if price < lower:  # Price below lower band - BUY signal
                self.buy()
            elif price > upper:  # Price above upper band - SELL signal
                self.sell()
    
    def notify_trade(self, trade):
        """Track completed trades"""
        if trade.isclosed:
            self.trade_count += 1
            # Calculate profit in pips (approximate)
            pnl_pips = trade.pnl / trade.value * PIP_VALUE if trade.value else 0
            self.profit_pips += pnl_pips
            
            if trade.pnl > 0:
                status = "WIN"
            else:
                status = "LOSS"
                
            print(f"Trade #{self.trade_count}: {status} - PnL: {trade.pnl:.2f} (~{pnl_pips:.1f} pips)")

print("✅ Bollinger Bands Strategy class defined")

✅ Bollinger Bands Strategy class defined


## Backtest Execution

In [24]:
# Create and run backtest with backtrader
print("🚀 Starting backtest with Bollinger Bands strategy...")
print(f"📊 Data points: {len(data):,}")
print(f"💰 Initial capital: ${INITIAL_CAPITAL:,}")
print(f"📊 Commission: {COMMISSION_PIPS} pips per trade")

# Create a backtrader Cerebro engine
cerebro = bt.Cerebro()

# Add strategy
cerebro.addstrategy(BollingerBandsStrategy)

# Create data feed from pandas DataFrame
data_feed = bt.feeds.PandasData(dataname=data)
cerebro.adddata(data_feed)

# Set initial capital
cerebro.broker.setcash(INITIAL_CAPITAL)

# Set commission (5 pips spread)
# For EURCHF around 0.93, 5 pips = 0.0005
commission_per_trade = COMMISSION_PIPS / PIP_VALUE
cerebro.broker.setcommission(commission=commission_per_trade)

# Add a PercentSizer to invest 100% of available cash
cerebro.addsizer(bt.sizers.PercentSizer, percents=100)

# Add analyzers for performance metrics
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='time_return')

print(f"✅ Backtrader engine configured")

# Run backtest
import time
start_time = time.time()
results = cerebro.run()
execution_time = time.time() - start_time

print(f"✅ Backtest completed in {execution_time:.2f} seconds")
print("=" * 60)

🚀 Starting backtest with Bollinger Bands strategy...
📊 Data points: 28,739
💰 Initial capital: $10,000
📊 Commission: 5 pips per trade
✅ Backtrader engine configured
✅ Bollinger Bands initialized: Period=1000, DevFactor=2
Trade #1: WIN - PnL: 14.94 (~0.0 pips)
Trade #2: WIN - PnL: 5.90 (~0.0 pips)
Trade #3: WIN - PnL: 7.61 (~0.0 pips)
Trade #4: WIN - PnL: 8.83 (~0.0 pips)
Trade #5: WIN - PnL: 6.59 (~0.0 pips)
Trade #6: WIN - PnL: 6.92 (~0.0 pips)
Trade #7: WIN - PnL: 7.72 (~0.0 pips)
Trade #8: LOSS - PnL: -23.79 (~0.0 pips)
Trade #9: LOSS - PnL: -59.89 (~0.0 pips)
Trade #10: LOSS - PnL: -21.19 (~0.0 pips)
Trade #11: WIN - PnL: 18.84 (~0.0 pips)
Trade #12: WIN - PnL: 6.53 (~0.0 pips)
Trade #13: WIN - PnL: 10.93 (~0.0 pips)
Trade #14: WIN - PnL: 10.61 (~0.0 pips)
Trade #15: WIN - PnL: 5.70 (~0.0 pips)
Trade #16: WIN - PnL: 7.61 (~0.0 pips)
Trade #17: WIN - PnL: 5.56 (~0.0 pips)
Trade #18: WIN - PnL: 4.10 (~0.0 pips)
✅ Backtest completed in 7.40 seconds


## Results Analysis

In [25]:
# Extract and display results
strat = results[0]

# Get analyzer results
trade_analyzer = strat.analyzers.trade_analyzer.get_analysis()
sharpe_ratio = strat.analyzers.sharpe.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
returns = strat.analyzers.returns.get_analysis()

# Calculate key metrics
final_value = cerebro.broker.getvalue()
total_return = (final_value - INITIAL_CAPITAL) / INITIAL_CAPITAL * 100

print("📊 BACKTEST RESULTS")
print("=" * 60)

# Portfolio metrics
print(f"💰 Initial Capital: ${INITIAL_CAPITAL:,}")
print(f"💰 Final Portfolio Value: ${final_value:,.2f}")
print(f"📊 Total Return: {total_return:.2f}%")
print(f"📊 Total PnL: ${final_value - INITIAL_CAPITAL:,.2f}")

# Trade statistics
if 'total' in trade_analyzer:
    total_trades = trade_analyzer['total']['total']
    won_trades = trade_analyzer['won']['total'] if 'won' in trade_analyzer else 0
    lost_trades = trade_analyzer['lost']['total'] if 'lost' in trade_analyzer else 0
    win_rate = (won_trades / total_trades * 100) if total_trades > 0 else 0
    
    print(f"\n📊 TRADE STATISTICS:")
    print(f"📊 Total Trades: {total_trades}")
    print(f"✅ Winning Trades: {won_trades}")
    print(f"❌ Losing Trades: {lost_trades}")
    print(f"🎯 Win Rate: {win_rate:.1f}%")
    
    if 'won' in trade_analyzer and 'pnl' in trade_analyzer['won']:
        avg_win = trade_analyzer['won']['pnl']['average']
        print(f"📊 Average Winning Trade: ${avg_win:.2f}")
        
    if 'lost' in trade_analyzer and 'pnl' in trade_analyzer['lost']:
        avg_loss = trade_analyzer['lost']['pnl']['average']
        print(f"📊 Average Losing Trade: ${avg_loss:.2f}")
        
    # Calculate profit factor
    if 'won' in trade_analyzer and 'lost' in trade_analyzer:
        gross_profit = trade_analyzer['won']['pnl']['total'] if 'pnl' in trade_analyzer['won'] else 0
        gross_loss = abs(trade_analyzer['lost']['pnl']['total']) if 'pnl' in trade_analyzer['lost'] else 1
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else 0
        print(f"📊 Profit Factor: {profit_factor:.2f}")
        
        # Estimate total pips (approximate)
        avg_price = data['close'].mean()
        total_pips = (final_value - INITIAL_CAPITAL) / avg_price * PIP_VALUE
        avg_pips_per_trade = total_pips / total_trades if total_trades > 0 else 0
        print(f"📊 Estimated Total PnL: {total_pips:.1f} pips")
        print(f"📊 Estimated Avg PnL/Trade: {avg_pips_per_trade:.2f} pips")

# Performance metrics
if sharpe_ratio:
    print(f"\n📈 PERFORMANCE METRICS:")
    if 'sharperatio' in sharpe_ratio and sharpe_ratio['sharperatio'] is not None:
        print(f"📊 Sharpe Ratio: {sharpe_ratio['sharperatio']:.3f}")
    else:
        print(f"📊 Sharpe Ratio: N/A")

if drawdown:
    if 'max' in drawdown and 'drawdown' in drawdown['max']:
        max_dd = drawdown['max']['drawdown']
        print(f"📉 Max Drawdown: {max_dd:.2f}%")

print("=" * 60)

📊 BACKTEST RESULTS
💰 Initial Capital: $10,000
💰 Final Portfolio Value: $9,844.85
📊 Total Return: -1.55%
📊 Total PnL: $-155.15

📊 TRADE STATISTICS:
📊 Total Trades: 18
✅ Winning Trades: 4
❌ Losing Trades: 14
🎯 Win Rate: 22.2%
📊 Average Winning Trade: $3.94
📊 Average Losing Trade: $-12.21
📊 Profit Factor: 0.09
📊 Estimated Total PnL: -1648494.8 pips
📊 Estimated Avg PnL/Trade: -91583.04 pips

📈 PERFORMANCE METRICS:
📊 Sharpe Ratio: N/A
📉 Max Drawdown: 1.97%


## Performance Visualization

In [26]:
# Plot results using matplotlib (extract equity curve)
print("📈 Generating equity curve plot...")

# Extract portfolio values from the strategy
strat = results[0]
time_return = strat.analyzers.time_return.get_analysis()

# Convert to pandas series for easier plotting
if time_return:
    equity_curve = pd.Series(time_return)
    equity_curve.index = pd.to_datetime(equity_curve.index)
    
    # Create equity curve plot
    plt.figure(figsize=PLOT_FIGURES_SIZE)
    
    # Plot equity curve
    plt.subplot(2, 1, 1)
    plt.plot(equity_curve.index, equity_curve.values * INITIAL_CAPITAL, 'b-', linewidth=2, label='Portfolio Value')
    plt.axhline(y=INITIAL_CAPITAL, color='r', linestyle='--', alpha=0.7, label='Initial Capital')
    plt.title('📈 Portfolio Equity Curve', fontsize=14, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Calculate and plot drawdown
    plt.subplot(2, 1, 2)
    running_max = (equity_curve * INITIAL_CAPITAL).expanding().max()
    drawdown_series = ((equity_curve * INITIAL_CAPITAL) - running_max) / running_max * 100
    plt.fill_between(drawdown_series.index, drawdown_series.values, 0, alpha=0.3, color='red')
    plt.plot(drawdown_series.index, drawdown_series.values, 'r-', linewidth=1)
    plt.title('📉 Drawdown (%)', fontsize=12, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Drawdown (%)')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print equity curve statistics
    print(f"\n📊 EQUITY CURVE STATISTICS:")
    print(f"📊 Starting Value: ${INITIAL_CAPITAL:,.2f}")
    print(f"📊 Ending Value: ${equity_curve.iloc[-1] * INITIAL_CAPITAL:,.2f}")
    print(f"📊 Peak Value: ${(equity_curve * INITIAL_CAPITAL).max():,.2f}")
    print(f"📊 Lowest Value: ${(equity_curve * INITIAL_CAPITAL).min():,.2f}")
    max_dd_pct = drawdown_series.min()
    print(f"📊 Maximum Drawdown: {max_dd_pct:.2f}%")
    
else:
    print("❌ No equity curve data available")
    
    # Fallback: use backtrader's built-in plotting
    print("📈 Using backtrader's built-in plotting...")
    plt.rcParams['figure.figsize'] = PLOT_FIGURES_SIZE
    cerebro.plot(style='candlestick', barup='green', bardown='red')

📈 Generating equity curve plot...


<IPython.core.display.Javascript object>


📊 EQUITY CURVE STATISTICS:
📊 Starting Value: $10,000.00
📊 Ending Value: $-5.84
📊 Peak Value: $9.74
📊 Lowest Value: $-71.65
📊 Maximum Drawdown: -inf%


In [27]:
# Create custom equity curve plot by tracking portfolio values
print("📈 Creating custom equity curve plot...")

# Extract equity values from the previous backtest
# We'll use a simpler approach by manually tracking portfolio value changes
print("🔄 Re-running backtest with detailed portfolio tracking...")

# Create a simple strategy that tracks portfolio values
class EquityTrackingStrategy(bt.Strategy):
    params = (
        ('bb_period', BB_WINDOW),
        ('bb_devfactor', BB_STD),
    )
    
    def __init__(self):
        self.bband = btind.BollingerBands(
            self.data.close, 
            period=self.params.bb_period,
            devfactor=self.params.bb_devfactor
        )
        self.upper_band = self.bband.lines.top
        self.middle_band = self.bband.lines.mid  
        self.lower_band = self.bband.lines.bot
        
        # Track portfolio values
        self.portfolio_values = []
        self.dates = []
    
    def next(self):
        # Record portfolio value and date
        current_value = self.broker.getvalue()
        self.portfolio_values.append(current_value)
        self.dates.append(self.data.datetime.datetime())
        
        price = self.data.close[0]
        upper = self.upper_band[0]
        lower = self.lower_band[0]
        middle = self.middle_band[0]
        
        if len(self.data) < self.params.bb_period:
            return
        
        # Exit conditions
        if self.position:
            if (self.position.size > 0 and price >= middle) or \
               (self.position.size < 0 and price <= middle):
                self.close()
        
        # Entry conditions
        elif not self.position:
            if price < lower:
                self.buy()
            elif price > upper:
                self.sell()

# Create new cerebro for equity tracking
cerebro_equity = bt.Cerebro()
cerebro_equity.addstrategy(EquityTrackingStrategy)

data_feed_equity = bt.feeds.PandasData(dataname=data)
cerebro_equity.adddata(data_feed_equity)

cerebro_equity.broker.setcash(INITIAL_CAPITAL)
cerebro_equity.broker.setcommission(commission=commission_per_trade)
cerebro_equity.addsizer(bt.sizers.PercentSizer, percents=100)

# Run backtest
start_time = time.time()
results_equity = cerebro_equity.run()
execution_time = time.time() - start_time

print(f"✅ Equity tracking completed in {execution_time:.2f} seconds")

# Extract portfolio values from strategy
strat_equity = results_equity[0]
portfolio_values = strat_equity.portfolio_values
portfolio_dates = strat_equity.dates

print(f"📊 Tracked {len(portfolio_values):,} portfolio values")

# Create equity curve plot
plt.figure(figsize=PLOT_FIGURES_SIZE)

# Main equity curve
plt.subplot(2, 1, 1)
plt.plot(portfolio_dates, portfolio_values, 'b-', linewidth=2, label='Portfolio Value')
plt.axhline(y=INITIAL_CAPITAL, color='r', linestyle='--', alpha=0.7, label='Initial Capital')
plt.title('📈 Portfolio Equity Curve (100% Investment per Trade)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True, alpha=0.3)

# Format y-axis to show currency
ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Calculate and plot drawdown
plt.subplot(2, 1, 2)
equity_series = pd.Series(portfolio_values, index=portfolio_dates)
running_max = equity_series.expanding().max()
drawdown_series = ((equity_series - running_max) / running_max * 100).fillna(0)
plt.fill_between(drawdown_series.index, drawdown_series.values, 0, alpha=0.3, color='red')
plt.plot(drawdown_series.index, drawdown_series.values, 'r-', linewidth=1)
plt.title('📉 Portfolio Drawdown (%)', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Drawdown (%)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate detailed statistics
print(f"\n📊 DETAILED EQUITY CURVE STATISTICS:")
print(f"📊 Starting Value: ${portfolio_values[0]:,.2f}")
print(f"📊 Ending Value: ${portfolio_values[-1]:,.2f}")
print(f"📊 Peak Value: ${max(portfolio_values):,.2f}")
print(f"📊 Lowest Value: ${min(portfolio_values):,.2f}")
total_return = (portfolio_values[-1] / portfolio_values[0] - 1) * 100
print(f"📊 Total Return: {total_return:.2f}%")
max_dd = drawdown_series.min()
print(f"📊 Maximum Drawdown: {max_dd:.2f}%")

# Calculate volatility
returns = equity_series.pct_change().dropna()
daily_vol = returns.std()
annual_vol = daily_vol * np.sqrt(252) * 100  # Annualized assuming 252 trading days
print(f"📊 Daily Volatility: {daily_vol * 100:.3f}%")
print(f"📊 Estimated Annual Volatility: {annual_vol:.2f}%")

# Calmar ratio (annual return / max drawdown)
if max_dd < 0:
    calmar_ratio = (total_return * 252 / len(returns)) / abs(max_dd)
    print(f"📊 Calmar Ratio: {calmar_ratio:.3f}")

print(f"📊 Number of Portfolio Updates: {len(portfolio_values):,}")

📈 Creating custom equity curve plot...
🔄 Re-running backtest with detailed portfolio tracking...
✅ Equity tracking completed in 6.38 seconds
📊 Tracked 27,740 portfolio values


<IPython.core.display.Javascript object>


📊 DETAILED EQUITY CURVE STATISTICS:
📊 Starting Value: $10,000.00
📊 Ending Value: $9,844.85
📊 Peak Value: $10,009.89
📊 Lowest Value: $9,812.35
📊 Total Return: -1.55%
📊 Maximum Drawdown: -1.97%
📊 Daily Volatility: 0.006%
📊 Estimated Annual Volatility: 0.09%
📊 Calmar Ratio: -0.007
📊 Number of Portfolio Updates: 27,740


## Trade Analysis

In [28]:
# Detailed trade analysis with backtrader results
print(f"📊 TRADE ANALYSIS")

if 'total' in trade_analyzer and trade_analyzer['total']['total'] > 0:
    total_trades = trade_analyzer['total']['total']
    print(f"📊 Total trades executed: {total_trades}")
    
    # Create a simple trade summary
    trades_data = []
    
    # Extract trade details if available
    if 'won' in trade_analyzer:
        won_trades = trade_analyzer['won']['total'] if 'total' in trade_analyzer['won'] else 0
        if 'pnl' in trade_analyzer['won']:
            avg_win_pnl = trade_analyzer['won']['pnl']['average']
            max_win_pnl = trade_analyzer['won']['pnl']['max']
            total_win_pnl = trade_analyzer['won']['pnl']['total']
        else:
            avg_win_pnl = max_win_pnl = total_win_pnl = 0
    else:
        won_trades = avg_win_pnl = max_win_pnl = total_win_pnl = 0
        
    if 'lost' in trade_analyzer:
        lost_trades = trade_analyzer['lost']['total'] if 'total' in trade_analyzer['lost'] else 0
        if 'pnl' in trade_analyzer['lost']:
            avg_loss_pnl = trade_analyzer['lost']['pnl']['average']
            max_loss_pnl = trade_analyzer['lost']['pnl']['max']
            total_loss_pnl = trade_analyzer['lost']['pnl']['total']
        else:
            avg_loss_pnl = max_loss_pnl = total_loss_pnl = 0
    else:
        lost_trades = avg_loss_pnl = max_loss_pnl = total_loss_pnl = 0
    
    # Convert to pips (approximate)
    avg_price = data['close'].mean()
    
    print(f"\n📊 TRADE STATISTICS:")
    print(f"✅ Winning trades: {won_trades} ({won_trades/total_trades:.1%})")
    print(f"❌ Losing trades: {lost_trades} ({lost_trades/total_trades:.1%})")
    
    if won_trades > 0:
        avg_win_pips = avg_win_pnl / avg_price * PIP_VALUE
        max_win_pips = max_win_pnl / avg_price * PIP_VALUE
        print(f"📊 Avg winning trade: ${avg_win_pnl:.2f} (~{avg_win_pips:.2f} pips)")
        print(f"📊 Max winning trade: ${max_win_pnl:.2f} (~{max_win_pips:.2f} pips)")
    
    if lost_trades > 0:
        avg_loss_pips = avg_loss_pnl / avg_price * PIP_VALUE
        max_loss_pips = max_loss_pnl / avg_price * PIP_VALUE
        print(f"📊 Avg losing trade: ${avg_loss_pnl:.2f} (~{avg_loss_pips:.2f} pips)")
        print(f"📊 Max losing trade: ${max_loss_pnl:.2f} (~{max_loss_pips:.2f} pips)")
    
    # Calculate average trade duration (estimate based on data length)
    total_duration_hours = (data.index[-1] - data.index[0]).total_seconds() / 3600
    avg_duration_per_trade = total_duration_hours / total_trades if total_trades > 0 else 0
    print(f"📊 Estimated avg trade duration: {avg_duration_per_trade:.2f} hours")
    
    total_pnl_pips = (final_value - INITIAL_CAPITAL) / avg_price * PIP_VALUE
    print(f"📊 Total PnL: ~{total_pnl_pips:.1f} pips")
    
    # Create a visualization with available data
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=PLOT_FIGURES_SIZE)
    fig.suptitle('🚀 Trade Analysis - Backtrader Results', fontsize=14, fontweight='bold')
    
    # Trade type distribution
    labels = ['Winning Trades', 'Losing Trades']
    sizes = [won_trades, lost_trades]
    colors = ['green', 'red']
    ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax1.set_title('📊 Trade Outcome Distribution')
    
    # PnL comparison
    categories = ['Avg Win', 'Avg Loss', 'Max Win', 'Max Loss']
    values = [avg_win_pnl, avg_loss_pnl, max_win_pnl, max_loss_pnl]
    bar_colors = ['green', 'red', 'darkgreen', 'darkred']
    bars = ax2.bar(categories, values, color=bar_colors, alpha=0.7)
    ax2.set_title('💰 PnL Comparison ($)')
    ax2.set_ylabel('PnL ($)')
    ax2.axhline(0, color='black', linestyle='--', alpha=0.5)
    
    # Rotate x-axis labels
    plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
    
    # Portfolio value progression (simulated based on total return)
    portfolio_values = [INITIAL_CAPITAL + (final_value - INITIAL_CAPITAL) * i / 100 for i in range(101)]
    ax3.plot(portfolio_values, 'b-', linewidth=2)
    ax3.set_title('💰 Portfolio Value Progression (Simulated)')
    ax3.set_xlabel('Progress (%)')
    ax3.set_ylabel('Portfolio Value ($)')
    ax3.grid(True, alpha=0.3)
    
    # Risk-reward analysis
    risk_reward_ratio = abs(avg_win_pnl / avg_loss_pnl) if avg_loss_pnl != 0 else 0
    win_rate_pct = won_trades / total_trades * 100 if total_trades > 0 else 0
    
    ax4.bar(['Win Rate %', 'Risk/Reward'], [win_rate_pct, risk_reward_ratio], 
            color=['blue', 'orange'], alpha=0.7)
    ax4.set_title('📊 Risk/Reward Analysis')
    ax4.set_ylabel('Value')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ No trades were executed")

📊 TRADE ANALYSIS
📊 Total trades executed: 18

📊 TRADE STATISTICS:
✅ Winning trades: 4 (22.2%)
❌ Losing trades: 14 (77.8%)
📊 Avg winning trade: $3.94 (~41912.12 pips)
📊 Max winning trade: $9.00 (~95587.68 pips)
📊 Avg losing trade: $-12.21 (~-129724.52 pips)
📊 Max losing trade: $-69.87 (~-742395.75 pips)
📊 Estimated avg trade duration: 34.67 hours
📊 Total PnL: ~-1648494.8 pips


<IPython.core.display.Javascript object>

## Strategy Comparison Summary

In [29]:
print("📊 STRATEGY IMPLEMENTATION COMPARISON")
print("=" * 60)
print("This notebook implements the same Bollinger Bands strategy using the")
print("backtrader library for validation and comparison purposes.")
print("")
print("🎯 Strategy Rules:")
print(f"   • Entry: Long when close < lower band, Short when close > upper band")
print(f"   • Exit: Close position when price reaches middle band")
print(f"   • BB Parameters: {BB_WINDOW} periods, {BB_STD} std dev")
print(f"   • Commission: {COMMISSION_PIPS} pips per trade")
print(f"   • Data: EURCHF tick data resampled to 1-minute bars")
print("")
print("📈 Key Results:")
print(f"   • Total Return: {total_return:.2f}%")
print(f"   • Final Portfolio Value: ${final_value:,.2f}")
if 'total' in trade_analyzer:
    total_trades = trade_analyzer['total']['total']
    won_trades = trade_analyzer['won']['total'] if 'won' in trade_analyzer else 0
    win_rate = (won_trades / total_trades * 100) if total_trades > 0 else 0
    print(f"   • Number of Trades: {total_trades}")
    print(f"   • Win Rate: {win_rate:.1f}%")
    
    if 'won' in trade_analyzer and 'lost' in trade_analyzer:
        gross_profit = trade_analyzer['won']['pnl']['total'] if 'pnl' in trade_analyzer['won'] else 0
        gross_loss = abs(trade_analyzer['lost']['pnl']['total']) if 'pnl' in trade_analyzer['lost'] else 1
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else 0
        print(f"   • Profit Factor: {profit_factor:.2f}")

if drawdown and 'max' in drawdown and 'drawdown' in drawdown['max']:
    max_dd = drawdown['max']['drawdown']
    print(f"   • Max Drawdown: {max_dd:.2f}%")

if sharpe_ratio and 'sharperatio' in sharpe_ratio and sharpe_ratio['sharperatio'] is not None:
    print(f"   • Sharpe Ratio: {sharpe_ratio['sharperatio']:.3f}")

print("="*60)

📊 STRATEGY IMPLEMENTATION COMPARISON
This notebook implements the same Bollinger Bands strategy using the
backtrader library for validation and comparison purposes.

🎯 Strategy Rules:
   • Entry: Long when close < lower band, Short when close > upper band
   • Exit: Close position when price reaches middle band
   • BB Parameters: 1000 periods, 2 std dev
   • Commission: 5 pips per trade
   • Data: EURCHF tick data resampled to 1-minute bars

📈 Key Results:
   • Total Return: -1.55%
   • Final Portfolio Value: $9,844.85
   • Number of Trades: 18
   • Win Rate: 22.2%
   • Profit Factor: 0.09
   • Max Drawdown: 1.97%
